In [178]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor

In [179]:
data = pd.read_json('/kaggle/input/netherlands-rent-properties/properties.json', lines=True)
data

,_id,externalId,areaRaw,areaSqm,city,coverImageUrl,crawlStatus,crawledAt,datesPublished,firstSeenAt,furnish,lastSeenAt,latitude,longitude,postalCode,postedAgo,propertyType,rawAvailability,rent,rentDetail,rentRaw,source,title,url,additionalCosts,additionalCostsRaw,deposit,depositRaw,descriptionNonTranslated,descriptionNonTranslatedRaw,descriptionTranslated,descriptionTranslatedRaw,detailsCrawledAt,energyLabel,gender,internet,isRoomActive,kitchen,living,matchAge,matchAgeBackup,matchCapacity,matchGender,matchGenderBackup,matchLanguages,matchStatus,matchStatusBackup,pageDescription,pageTitle,pets,registrationCost,registrationCostRaw,roommates,shower,smokingInside,toilet,userDisplayName,userId,userLastLoggedOn,userMemberSince,userPhotoUrl,additionalCostsDescription
0,{'$oid': '5d2b113a43cbfd7c77a998f4'},room-1686123,14 m2,14,Rotterdam,https://resources.kamernet.nl/image/913b4b03-5...,done,{'$date': '2019-07-26T22:18:23.018+0000'},"[{'$date': '2019-07-14T11:25:46.511+0000'}, {'...",{'$date': '2019-07-14T11:25:46.511+0000'},Unfurnished,{'$date': '2019-07-26T22:18:23.142+0000'},51.896601,4.514993,3074HN,4w,Room,26-06-'19 - Indefinite period,500,,"€ 500,-",kamernet,West-Varkenoordseweg,https://kamernet.nl/en/for-rent/room-rotterdam...,50.0,\n € 50\n ...,500.0,\n € 500\n ...,"Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...","Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...",{'$date': '2019-07-22T07:10:41.849+0000'},Unknown,Mixed,Yes,true,Shared,None,16 years -\n 99 years,16 years -\n 99 years,1 person,Not important,Not important,Not important,Not important,Not important,"Room for rent in Rotterdam, West-Varkenoordse...",Room for rent in Rotterdam €500 | Kamernet,No,0,\n € 0\n ...,5,Shared,No,Shared,Huize west,4680711.0,21-07-2019,26-06-2019,https://resources.kamernet.nl/Content/images/s...,NaN
1,{'$oid': '5d2b113a43cbfd7c77a9991a'},studio-1691193,30 m2,30,Amsterdam,https://resources.kamernet.nl/image/5e11d6b5-8...,done,{'$date': '2019-08-10T22:28:46.099+0000'},"[{'$date': '2019-07-14T11:25:46.677+0000'}, {'...",{'$date': '2019-07-14T11:25:46.677+0000'},Furnished,{'$date': '2019-08-10T22:28:46.229+0000'},52.370200,4.920721,1018AS,4w,Studio,15-08-'19 - Indefinite period,950,Utilities incl.,"€ 950,- Utilities incl.",kamernet,Parelstraat,https://kamernet.nl/en/for-rent/studio-amsterd...,0.0,\n € 0\n ...,895.0,\n € 895\n ...,"Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...","Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...",{'$date': '2019-07-22T06:29:33.112+0000'},Unknown,Unknown,Yes,true,Own,Own,18 years -\n 99 years,18 years -\n 99 years,1 person,Not important,Not important,Not important,"Working student, Working","Working student, Working","Studio for rent in Amsterdam, Parelstraat, fo...",Studio for rent in Amsterdam €950 | Kamernet,No,0,\n € 0\n ...,None,Own,No,Own,Cor,1865530.0,20-07-2019,05-01-2012,https://resources.kamernet.nl/Content/images/p...,NaN
2,{'$oid': '5d2b113a43cbfd7c77a99931'},room-1690545,11 m2,11,Amsterdam,https://resources.kamernet.nl/image/74b93a27-a...,done,{'$date': '2019-10-02T22:00:33.141+0000'},"[{'$date': '2019-07-14T11:25:46.834+0000'}, {'...",{'$date': '2019-07-14T11:25:46.834+0000'},Furnished,{'$date': '2019-10-02T22:00:33.264+0000'},52.350880,4.854786,1075SB,09 Jul,Room,01-08-'19 - Indefinite period,1000,Utilities incl.,"€ 1000,- Utilities incl.",kamernet,Zeilstraat,https://kamernet.nl/en/for-rent/room-amsterdam...,NaN,\n -\n ...,1000.0,\n € 1000\n ...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e...,{'$date': '2019-07-21T08:44:32.816+0000'},Unknown,Mixed,Yes,true,Shared,Shared,16 years -\n 93 years,16 years -\n 93 years,1 person,Not important,Not i

In [180]:
data['crawlStatus'].value_counts()

done           46622
unavailable      100
Name: crawlStatus, dtype: int64

In [181]:
data.query("crawlStatus == 'unavailable'").index

Int64Index([   77,    80,    91,   116,   231,   348,   484,   492,   537,
              553,   555,   570,   626,   740,   749,   840,   907,   930,
              932,   999,  1089,  1122,  1192,  1271,  1337,  1350,  1352,
             1565,  1615,  1942,  2038,  2346,  2598,  2704,  2995,  3041,
             3372,  3373,  3409,  3423,  3427,  3446,  3461,  3462,  3463,
             3488,  4176,  4202,  4213,  4342,  4347,  4398,  4443,  4482,
             4514,  4552,  4583,  4648,  4869,  4925,  4932,  4972,  5368,
             5378,  5641,  6019,  6020,  6275,  6361,  6493,  6643,  6768,
             6834,  6840,  6893,  6932,  6963,  6967,  8895, 10069, 11904,
            13503, 15265, 15739, 22198, 24546, 26579, 27214, 27377, 27378,
            27823, 34340, 35411, 38851, 42950, 43453, 44914, 45434, 45492,
            45520],
           dtype='int64')

In [182]:
def preprocess_inputs(df):
    df = df.copy()
    
    ## dropping the crawlStatus == unavailable
    bad_rows = df.query("crawlStatus == 'unavailable'").index
    df = df.drop(bad_rows, axis = 0).reset_index(drop=True)
    
    ## Selecting the features
    df = df[['areaSqm', 'city', 'furnish', 'latitude', 'longitude', 'propertyType', 'rent', 'internet', 'kitchen', 'living', 'pets', 'shower', 'smokingInside', 'toilet']]
    
    ## Encode imporper values
    df = df.replace({'': np.NaN, 'Unknown': np.NaN})
    
    ## Fill missing values
    missing_value_columns = df.columns[df.isna().sum() > 0]
    for column in missing_value_columns:
        df[column] = df[column].fillna(df[column].mode()[0])
        
    ## Split the df into X and y
    y = df['rent']
    X = df.drop('rent', axis = 1)
    
    ## Train_test_split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, shuffle = True, random_state = 1)
    
    return X_train, X_test, y_train, y_test, X
    

In [183]:
X_train, X_test, y_train, y_test, X = preprocess_inputs(data)
X_train

,areaSqm,city,furnish,latitude,longitude,propertyType,internet,kitchen,living,pets,shower,smokingInside,toilet
29791,61,Rotterdam,Unfurnished,51.925125,4.486212,Apartment,Yes,Shared,Shared,No,Shared,No,Shared
44827,45,Rotterdam,Unfurnished,51.893369,4.517075,Apartment,Yes,Own,Own,No,Own,No,Own
37089,105,Amsterdam,Furnished,52.376979,4.839116,Apartment,Yes,Own,Own,No,Own,No,Own
13269,20,Delft,Uncarpeted,51.996010,4.352954,Room,Yes,Shared,Shared,By mutual agreement,Shared,Not important,Shared
14654,44,Rotterdam,Furnished,51.891709,4.480317,Apartment,Yes,Shared,Shared,No,Shared,No,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43723,11,Enschede,Unfurnished,52.234413,6.849043,Room,Yes,Shared,Shared,No,Shared,No,Shared
32511,21,Utrecht,Furnished,52.086425,5.125311,Room,Yes,Shared,Shared,No,Shared,No,Shared
5192,16,Groningen,Furnished,53.229623,6.524759,Room,Yes,Shared,Shared,No,Shared,No,Shared
12172,7,Utrecht,Furnished,52.102119,5.096010,Room,Yes,Shared,None,No,Shared,No,Shared


In [184]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46622 entries, 0 to 46621
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   areaSqm        46622 non-null  int64  
 1   city           46622 non-null  object 
 2   furnish        46622 non-null  object 
 3   latitude       46622 non-null  float64
 4   longitude      46622 non-null  float64
 5   propertyType   46622 non-null  object 
 6   internet       46622 non-null  object 
 7   kitchen        46622 non-null  object 
 8   living         46622 non-null  object 
 9   pets           46622 non-null  object 
 10  shower         46622 non-null  object 
 11  smokingInside  46622 non-null  object 
 12  toilet         46622 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 4.6+ MB


In [185]:
# X.isna().sum()

In [186]:
X.columns

Index(['areaSqm', 'city', 'furnish', 'latitude', 'longitude', 'propertyType',
       'internet', 'kitchen', 'living', 'pets', 'shower', 'smokingInside',
       'toilet'],
      dtype='object')

In [187]:
# X.select_dtypes('object').columns

In [188]:
# len(X['city'].unique())

In [189]:
# {column: len(X[column].unique()) for column in X.select_dtypes('object').columns}

In [190]:
X.isna().mean()

areaSqm          0.0
city             0.0
furnish          0.0
latitude         0.0
longitude        0.0
propertyType     0.0
internet         0.0
kitchen          0.0
living           0.0
pets             0.0
shower           0.0
smokingInside    0.0
toilet           0.0
dtype: float64

In [191]:
# X.info()

In [192]:
# X.living.mode()

In [193]:
# X.living.mode()[0]

In [194]:
# X.isna().sum() > 0

In [195]:
# X.columns[X.isna().sum() > 0]

In [201]:
nominal_features = [
   'city', 'furnish', 'propertyType', 'kitchen', 'living', 'pets', 'shower', 'smokingInside', 'toilet'
]

binary_transformer = Pipeline(steps = [
    ('ordinal', OrdinalEncoder())
])

nominal_transformer = Pipeline(steps = [
    ('onehot', OneHotEncoder(sparse = False, handle_unknown = 'ignore'))
])

preprocessor = ColumnTransformer(transformers = [
    ('binary', binary_transformer, ['internet']),
    ('nominal', nominal_transformer, nominal_features)    
], remainder = 'passthrough')

model = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [202]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('binary',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['internet']),
                                                 ('nominal',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['city', 'furnish',
                                                   'propertyType', 'kitchen',
                                                   'living', 'pets', 'shower',
                                              

In [206]:
y_pred = model.predict(X_test)
y_pred

array([ 430.56      ,  977.59666667,  337.7225    , ..., 1295.93333333,
        346.076     ,  433.89      ])

In [205]:
y_test

31410     400
34454     775
42517     345
36306     500
43045     800
         ... 
42138    1600
36266     795
7926     1250
25058     365
15087     475
Name: rent, Length: 13987, dtype: int64

In [224]:
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
rmse

156.97008300041264

In [223]:
r2 = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
r2

0.8561424683356023

2395663115.600629